In [1]:
import pandas as pd
import numpy as np
import os
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib
matplotlib.rcParams['axes.unicode_minus'] = False
pd.options.mode.chained_assignment = None  # default='warn'
plt.rcParams['font.family'] = 'Malgun Gothic'
import datetime
import time
import math

In [2]:
def get_gu_dong(df):
    df = df.copy()
    
    df['구'] = df['지번주소'].apply(lambda x: x.split(' ')[1])
    df['동'] = df['지번주소'].apply(lambda x: x.split(' ')[2])
    
    return df

In [3]:
bulktrade_basedir = './국토교통부_실거래가_공개시스템/집값분석/일괄구매/10호이상/'

In [4]:
date_today = pd.to_datetime(datetime.datetime.now())
target_period = 6 # in months

In [5]:
concat_df = pd.read_csv(bulktrade_basedir+'2018년01월01일부터_2021년06월07일까지_SH_LH_아파트_연립다세대_오피스텔_거래사례.csv')
missing_df = pd.read_csv(bulktrade_basedir+'2018부터_아파트_연립다세대_오피스텔_일괄구매_전체_소유자_포함_기존자료_제외.csv')

In [6]:
concat_df = concat_df.drop(columns=['Unnamed: 0'])

In [7]:
print(concat_df.shape)
concat_df.head()

(14793, 13)


,지번주소,도로명,건물(단지)명,전용면적(㎡),거래금액(만원),층,건축년도,계약날짜기준_건물연식,계약날짜,단가(만원/㎡),건물연식,부동산유형,소유자
0,서울특별시 강남구 개포동 1195-10,논현로12길 3-1,동영빌,25.39,28300,3,2020.0,0.0,2020-03-02,1114.612052,1.0,연립다세대,SH
1,서울특별시 강남구 개포동 1195-10,논현로12길 3-1,동영빌,25.39,28300,2,2020.0,0.0,2020-03-02,1114.612052,1.0,연립다세대,SH
2,서울특별시 강남구 개포동 1195-10,논현로12길 3-1,동영빌,25.46,28400,3,2020.0,0.0,2020-03-02,1115.475255,1.0,연립다세대,SH
3,서울특별시 강남구 개포동 1195-10,논현로12길 3-1,동영빌,25.46,28400,2,2020.0,0.0,2020-03-02,1115.475255,1.0,연립다세대,SH
4,서울특별시 강남구 개포동 1195-10,논현로12길 3-1,동영빌,25.55,28500,3,2020.0,0.0,2020-03-02,1115.459883,1.0,연립다세대,SH


In [8]:
print(missing_df.shape)
missing_df.head()

(11112, 13)


,지번주소,도로명,건물(단지)명,전용면적(㎡),거래금액(만원),층,건축년도,계약날짜기준_건물연식,계약날짜,단가(만원/㎡),건물연식,부동산유형,소유자
0,서울특별시 강남구 개포동 1166-9,논현로2길 37,동영빌,21.50,25850,4,2021.0,0.0,2021-06-22,1202.325581,0.0,연립다세대,장**외 1명
1,서울특별시 강남구 개포동 1166-9,논현로2길 37,동영빌,21.92,25950,3,2021.0,0.0,2021-06-22,1183.850365,0.0,연립다세대,장**외 1명
2,서울특별시 강남구 개포동 1166-9,논현로2길 37,동영빌,21.92,26050,2,2021.0,0.0,2021-06-22,1188.412409,0.0,연립다세대,장**외 1명
3,서울특별시 강남구 개포동 1166-9,논현로2길 37,동영빌,24.42,28150,5,2021.0,0.0,2021-06-22,1152.743653,0.0,연립다세대,장**외 1명
4,서울특별시 강남구 개포동 1166-9,논현로2길 37,동영빌,25.01,29450,3,2021.0,0.0,2021-06-22,1177.528988,0.0,연립다세대,장**외 1명


In [9]:
missing_df['소유자'].isna().sum()

394

In [10]:
concat_df = pd.concat([concat_df, missing_df])

concat_df['계약날짜'] = pd.to_datetime(concat_df['계약날짜'])
#yunrip_bulk_df = yunrip_bulk_df[yunrip_bulk_df['계약날짜']  >= (date_today - pd.DateOffset(months=target_period))]
concat_df = concat_df[
    (concat_df['소유자'] == '서***~')
    |(concat_df['소유자'] == '한***~')
    |(concat_df['소유자'] == 'SH')
    |(concat_df['소유자'] == 'LH')
].reset_index(drop=True)
concat_df = get_gu_dong(concat_df)
print(concat_df.shape)
concat_df.head()

(15782, 15)


,지번주소,도로명,건물(단지)명,전용면적(㎡),거래금액(만원),층,건축년도,계약날짜기준_건물연식,계약날짜,단가(만원/㎡),건물연식,부동산유형,소유자,구,동
0,서울특별시 강남구 개포동 1195-10,논현로12길 3-1,동영빌,25.39,28300,3,2020.0,0.0,2020-03-02,1114.612052,1.0,연립다세대,SH,강남구,개포동
1,서울특별시 강남구 개포동 1195-10,논현로12길 3-1,동영빌,25.39,28300,2,2020.0,0.0,2020-03-02,1114.612052,1.0,연립다세대,SH,강남구,개포동
2,서울특별시 강남구 개포동 1195-10,논현로12길 3-1,동영빌,25.46,28400,3,2020.0,0.0,2020-03-02,1115.475255,1.0,연립다세대,SH,강남구,개포동
3,서울특별시 강남구 개포동 1195-10,논현로12길 3-1,동영빌,25.46,28400,2,2020.0,0.0,2020-03-02,1115.475255,1.0,연립다세대,SH,강남구,개포동
4,서울특별시 강남구 개포동 1195-10,논현로12길 3-1,동영빌,25.55,28500,3,2020.0,0.0,2020-03-02,1115.459883,1.0,연립다세대,SH,강남구,개포동


In [11]:
summary_df = pd.read_excel(bulktrade_basedir + '2018년부터/' + '2018년01월01일부터_2021년06월22일까지_SH_LH_아파트_연립다세대_오피스텔_거래사례_요약본' + '.xlsx')

In [12]:
print(summary_df.shape)
summary_df.head()

(864, 17)


,지번주소,건물(단지)명,건축년도,계약날짜,부동산유형,소유자,평균 전용면적(㎡),총 전용면적(㎡),전용면적 표준편차(㎡),평균 거래금액(만원),총 거래금액(만원),거래금액 표준편차(만원),최대 층,평균 단가(만원),단가 표준편차(만원),세대 수,Unnamed: 16
0,강남구 개포동 1237-7,한별2,2021,2021-06-01,오피스텔,SH,29.665000,118.66,0.390000,33565.250000,134261,437.500000,2,1131.477759,0.128995,4,NaN
1,강남구 개포동 1237-7,한별2,2021,2021-06-01,연립다세대,SH,29.946667,449.20,0.037544,38163.333333,572450,44.185755,6,1274.376854,0.358219,15,NaN
2,강남구 개포동 1216-7,백년빌,2020,2020-03-18,연립다세대,SH,26.686000,400.29,2.367074,30573.333333,458600,2134.651750,5,1147.330195,25.484846,15,NaN
3,강남구 개포동 1195-10,동영빌,2020,2020-03-02,연립다세대,SH,28.814667,432.22,4.855684,31463.333333,471950,4139.637090,5,1097.178201,36.299709,15,NaN
4,강남구 개포동 1199-7,개포백년빌,2019,2019-12-19,연립다세대,LH,27.905000,390.67,3.070186,28857.142857,404000,3536.722578,5,1033.367584,37.609677,14,NaN


In [13]:
summary_df['계약날짜'] = pd.to_datetime(summary_df['계약날짜'])

In [14]:
summary_df['계약날짜'].describe()

c:\users\max6296\ml\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  """Entry point for launching an IPython kernel.


count                     864
unique                    446
top       2020-12-28 00:00:00
freq                       11
first     2018-01-02 00:00:00
last      2021-06-22 00:00:00
Name: 계약날짜, dtype: object

In [ ]:
sales_idx_basedir = './한국부동산원/실거래가격지수/'
filenames = [f for f in os.listdir(sales_idx_basedir) if '연립다세대' in f]

sales_idx_df = pd.read_excel(sales_idx_basedir + filenames[0], header=[10, 11])

sales_idx_cols_list = []
for i in range(sales_idx_df.shape[1]):
    if i == 0:
        sales_idx_cols_list.append(sales_idx_df.columns[i][0])
    else:
        sales_idx_cols_list.append(sales_idx_df.columns[i][0] + '_' + sales_idx_df.columns[i][1])

sales_idx_df.columns = sales_idx_cols_list

In [ ]:
def get_sales_idx(x):
    # x is a pandas timestamp
    
    df = sales_idx_df.copy()
    
    if pd.isna(x) == True:
        return x
    else:
        year = str(x.year)
        month = str(x.month)
        if len(month) == 1:
            month = '0'+month
        
        idx = df[year+'년 '+month+'월_지수'].iloc[0]
        
        return idx

In [ ]:
def get_index_applied_prices(df):
    df = df.copy()
    
    last_idx = sales_idx_df.iloc[0,-2]
    
    last_date = sales_idx_df.columns[-1].split('_')[0]
    
    last_date = last_date.split(' ')[0].replace('년','') + '-' + last_date.split(' ')[1].replace('월','') + '-01'
    
    target_upper_limit_df = df[df['계약날짜'] < pd.to_datetime(last_date)]
    target_upper_limit_df['계약날짜_실거래지수'] = target_upper_limit_df['계약날짜'].apply(get_sales_idx)
    target_upper_limit_df['지수적용단가'] = target_upper_limit_df['단가(만원/㎡)'] / target_upper_limit_df['계약날짜_실거래지수'] * last_idx
    df['지수적용단가'] = df['단가(만원/㎡)']
    df.loc[target_upper_limit_df.index, '지수적용단가'] = target_upper_limit_df['지수적용단가']
    df['평가날짜'] = df['계약날짜']
    df.loc[target_upper_limit_df.index, '평가날짜'] = pd.to_datetime(last_date[:-2] + '01')
    
    return df

In [ ]:
concat_df = get_index_applied_prices(concat_df)
print(concat_df.shape)
concat_df.head()

In [ ]:
concat_df['구_동'] = concat_df['구'] + '_' + concat_df['동']

In [ ]:
def change_name(x):
    # x is a string
    
    if pd.isna(x) == True:
        return x
    else:
        if x == '한***~':
            return '한국토지주택공사'
        elif x == '서***~':
            return '서울주택도시공사'
        else:
            return x

In [ ]:
concat_df['소유자'] = concat_df['소유자'].apply(change_name)

In [ ]:
concat_df['지번주소'].unique()

In [ ]:
concat_df['소유자'].value_counts()

In [ ]:
concat_df.head()

In [ ]:
to_print_df = concat_df.drop(columns=['도로명', '계약날짜기준_건물연식', '건물연식', '구', '동', '지수적용단가', '평가날짜'])
print(to_print_df.shape)
to_print_df.head()

In [ ]:
to_print_df['구_동'].nunique()

In [ ]:
to_print_df = to_print_df[to_print_df['계약날짜'] >= pd.to_datetime('2018-01-01')]
print(to_print_df.shape)
to_print_df.head()

In [ ]:
to_print_df['구_동'].nunique()

In [ ]:
to_print_df['계약날짜'].describe()

In [ ]:
for gudong in tqdm(to_print_df['구_동'].unique()):
    gudong_df = to_print_df[to_print_df['구_동'] == gudong].drop(columns=['구_동'])
    gudong_df.to_excel(bulktrade_basedir + '2018년부터/' + gudong + '.xlsx', index=False)

In [ ]:
to_print_df.head()

In [ ]:
to_print_df['소유자'] = to_print_df['소유자'].apply(lambda x: x.replace('서울주택도시공사', 'SH'))
to_print_df['소유자'] = to_print_df['소유자'].apply(lambda x: x.replace('한국토지주택공사', 'LH'))

In [ ]:
to_print_df['소유자'].value_counts()

In [ ]:
to_print_df['평균_전용면적(㎡)'] = to_print_df[['지번주소', '건축년도', '부동산유형', '전용면적(㎡)']].groupby(['지번주소', '건축년도', '부동산유형']).transform('mean')['전용면적(㎡)']

In [ ]:
to_print_df['총_전용면적(㎡)'] = to_print_df[['지번주소', '건축년도', '부동산유형', '전용면적(㎡)']].groupby(['지번주소', '건축년도', '부동산유형']).transform('sum')['전용면적(㎡)']

In [ ]:
to_print_df['전용면적_표준편차(㎡)'] = to_print_df[['지번주소', '건축년도', '부동산유형', '전용면적(㎡)']].groupby(['지번주소', '건축년도', '부동산유형']).transform('std')['전용면적(㎡)']

In [ ]:
to_print_df['평균_거래금액(만원)'] = to_print_df[['지번주소', '건축년도', '부동산유형', '거래금액(만원)']].groupby(['지번주소', '건축년도', '부동산유형']).transform('mean')['거래금액(만원)']

In [ ]:
to_print_df['총_거래금액(만원)'] = to_print_df[['지번주소', '건축년도', '부동산유형', '거래금액(만원)']].groupby(['지번주소', '건축년도', '부동산유형']).transform('sum')['거래금액(만원)']

In [ ]:
to_print_df['거래금액_표준편차(만원)'] = to_print_df[['지번주소', '건축년도', '부동산유형', '거래금액(만원)']].groupby(['지번주소', '건축년도', '부동산유형']).transform('std')['거래금액(만원)']

In [ ]:
to_print_df['최대_층'] = to_print_df[['지번주소', '건축년도', '부동산유형', '층']].groupby(['지번주소', '건축년도', '부동산유형']).transform('max')['층']

In [ ]:
to_print_df['평균_단가(만원)'] = to_print_df[['지번주소', '건축년도', '부동산유형', '단가(만원/㎡)']].groupby(['지번주소', '건축년도', '부동산유형']).transform('mean')['단가(만원/㎡)']

In [ ]:
to_print_df['단가_표준편차(만원)'] = to_print_df[['지번주소', '건축년도', '부동산유형', '단가(만원/㎡)']].groupby(['지번주소', '건축년도', '부동산유형']).transform('std')['단가(만원/㎡)']

In [ ]:
to_print_df['세대_수'] = to_print_df[['지번주소', '건축년도', '부동산유형', '단가(만원/㎡)']].groupby(['지번주소', '건축년도', '부동산유형']).transform('count')['단가(만원/㎡)']

In [ ]:
to_print_df.head()

In [ ]:
to_print_df['계약날짜'].describe()

In [ ]:
to_print_df['계약날짜'].max()

In [ ]:
to_print_df['계약날짜'].nunique()

In [ ]:
idx_to_drop = []
for addr in tqdm(to_print_df['지번주소'].unique()):
    addr_df = to_print_df[to_print_df['지번주소'] == addr]
    
    if to_print_df['계약날짜'].nunique() > 1:
        not_max_df = addr_df[addr_df['계약날짜'] != addr_df['계약날짜'].max()]
        [idx_to_drop.append(i) for i in not_max_df.index]

In [ ]:
len(idx_to_drop)

In [ ]:
to_print_df = to_print_df.drop(idx_to_drop).reset_index(drop=True)

In [ ]:
summary_df = to_print_df.drop(columns=['전용면적(㎡)', '거래금액(만원)', '층', '단가(만원/㎡)', '구_동']).drop_duplicates().reset_index(drop=True)
print(summary_df.shape)
summary_df.head()

In [ ]:
summary_df['구_동'] = summary_df['지번주소'].apply(lambda x: x.split(' ')[1]) + '_' + summary_df['지번주소'].apply(lambda x: x.split(' ')[2])

In [ ]:
summary_df = summary_df.sort_values(['구_동', '계약날짜'], ascending=[True, False]).reset_index(drop=True)
print(summary_df.shape)
summary_df.head()

In [ ]:
summary_df = summary_df.drop(columns=['구_동'])

In [ ]:
summary_df['지번주소'] = summary_df['지번주소'].apply(lambda x: x.replace('서울특별시 ', ''))
summary_df['소유자'] = summary_df['소유자'].apply(lambda x: x.replace('서울주택도시공사', 'SH'))
summary_df['소유자'] = summary_df['소유자'].apply(lambda x: x.replace('한국토지주택공사', 'LH'))

In [ ]:
summary_df['계약날짜'].describe()

In [ ]:
summary_df.to_excel(bulktrade_basedir + '2018년부터/' + '2018년01월01일부터_2021년06월22일까지_SH_LH_아파트_연립다세대_오피스텔_거래사례_요약본' + '.xlsx', index=False)

In [ ]:
summary_df['지번주소'] = '서울특별시 ' + summary_df['지번주소']

In [ ]:
print_missing_df = summary_df[summary_df['지번주소'].isin(missing_df['지번주소'].unique())]
print(print_missing_df.shape)

In [ ]:
print_missing_df['지번주소'] = print_missing_df['지번주소'].apply(lambda x: x.replace('서울특별시 ', ''))

In [ ]:
print_missing_df.head()

In [ ]:
print_missing_df.to_excel(bulktrade_basedir + '빠진애들.xlsx', index=False)

In [ ]:
inv_df = concat_df[concat_df['소유자'] == '서울주택도시공사']
print(inv_df.shape)
inv_df.head()

In [ ]:
inv_df = inv_df.sort_values(['단가(만원/㎡)'], ascending=False)
print(inv_df.shape)
inv_df.head()

In [ ]:
land_specs_df = pd.read_csv('./prepped_data/land_specs_baseline.csv')
print(land_specs_df.shape)
land_specs_df.head()

In [ ]:
last_df = land_specs_df.drop_duplicates(subset=['지번주소'], keep='last').drop(columns=['년']).reset_index(drop=True)
print(last_df.shape)
last_df.head()

In [ ]:
merge_df = concat_df.merge(last_df, on=['지번주소'])
print(merge_df.shape)
merge_df.head()

In [ ]:
merge_df['도로접면'].value_counts()

In [ ]:
inv_idx = []
for addr in tqdm(concat_df['지번주소'].unique()):
    addr_df = concat_df[concat_df['지번주소'] == addr]
    
    if addr_df['부동산유형'].nunique() > 1:
        [inv_idx.append(i) for i in addr_df.index]

In [ ]:
len(inv_idx)

In [ ]:
yunrip_officetel_df = concat_df.loc[inv_idx]
print(yunrip_officetel_df.shape)
yunrip_officetel_df.head()

In [ ]:
yunrip_officetel_df['지번주소'].nunique()

In [ ]:
for addr in yunrip_officetel_df['지번주소'].unique():
    addr_df = yunrip_officetel_df[yunrip_officetel_df['지번주소'] == addr]
    
    addr_df = addr_df.drop(columns=['도로명', '계약날짜기준_건물연식', '건물연식', '구', '동', '지수적용단가', '평가날짜', '구_동'])
    
    addr_df.to_excel('./연립다세대_오피스텔_같이사례_주소별/' + addr + '.xlsx', index=False)
    
    print(addr_df.shape)
    display(addr_df)

In [ ]:
1131.413262 / 1274.182789

In [ ]:
889.092576 / 1324.609734

In [ ]:
972.300735 / 1370.087336

In [ ]:
1563.467492 / 1615.878107

In [ ]:
1114.762516 / 1222.272522

In [ ]:
648.957633 / 884.445951

In [ ]:
623.745819 / 689.285714

In [ ]:
yunrip_officetel_df.to_excel('SH_LH_거래사례중_연립다세대_오피스텔_같이_있는_사례들.xlsx')

In [ ]:
count = 0
for addr in tqdm(concat_df['구_동'].unique()):
    addr_df = concat_df[concat_df['구_동'] == addr]
    addr_df = addr_df[['지번주소', '건물(단지)명', '건축년도', '건물연식', '계약날짜기준_건물연식', '층', '전용면적(㎡)',
                       '계약날짜', '단가(만원/㎡)', '평가날짜', '지수적용단가', '소유자', '부동산유형'
                      ]]
    addr_df = addr_df.sort_values(['지번주소', '건축년도', '계약날짜', '층', '전용면적(㎡)'])
    addr_df.to_excel(bulktrade_basedir + '동별/' + addr + '.xlsx', index=False)
    
    
    
    
    count += 1

In [ ]:
concat_df['지번주소'].nunique()

In [ ]:
concat_df.head()

In [ ]:
addr_to_inv = []
for addr in tqdm(concat_df['지번주소'].unique()):
    addr_df = concat_df[concat_df['지번주소'] == addr]
    
    inv_30_df = addr_df[
        (addr_df['전용면적(㎡)'] >= 28)
        &(addr_df['전용면적(㎡)'] <= 32)
    ]
    
    inv_80_df = addr_df[
        (addr_df['전용면적(㎡)'] >= 80)
    ]
    
    if (inv_30_df.shape[0] > 1) & (inv_80_df.shape[0] > 1):
        addr_to_inv.append(addr)

In [ ]:
len(addr_to_inv)

In [ ]:
addr_to_inv = []
for addr in tqdm(concat_df['지번주소'].unique()):
    addr_df = concat_df[concat_df['지번주소'] == addr]

    inv_80_df = addr_df[
        (addr_df['전용면적(㎡)'] >= 80)
    ]
    
    if (inv_80_df.shape[0] > 1):
        addr_to_inv.append(addr)

In [ ]:
len(addr_to_inv)

In [ ]:
addr_to_inv

In [ ]:
inv_df = concat_df[concat_df['지번주소'].isin(addr_to_inv)]
print(inv_df.shape)
inv_df.head()

In [ ]:
inv_df

In [ ]:
concat_df['전용면적(㎡)'] = concat_df['전용면적(㎡)'].apply(round)

In [ ]:
concat_df['평균단가'] = concat_df[['지번주소', '건축년도', '전용면적(㎡)', '단가(만원/㎡)']].groupby(['지번주소', '건축년도', '전용면적(㎡)']).transform('mean')['단가(만원/㎡)']

In [ ]:
concat_df['지수적용평균단가'] = concat_df[['지번주소', '건축년도', '전용면적(㎡)', '지수적용단가']].groupby(['지번주소', '건축년도', '전용면적(㎡)']).transform('mean')['지수적용단가']

In [ ]:
concat_df = concat_df.drop(columns=['도로명', '거래금액(만원)', '층', '단가(만원/㎡)', '지수적용단가']).drop_duplicates().reset_index(drop=True)
print(concat_df.shape)
concat_df.head()

In [ ]:
yunrip_selected_df = yunrip_bulk_df[['지번주소', '건축년도', '건물연식', '계약날짜', '계약날짜기준_건물연식', '전용면적(㎡)', '지번별_전용면적별_평균단가', '부동산유형', '소유자']].drop_duplicates().reset_index(drop=True)
print(yunrip_selected_df.shape)
yunrip_selected_df.head()

In [ ]:
yunrip_selected_df = yunrip_selected_df.sort_values(['지번주소']).reset_index(drop=True)
yunrip_selected_df.to_excel(bulktrade_basedir + '최근_6개월_SH_LH_거래사례.xlsx', index=False)

In [ ]:
yunrip_selected_df['지번별_평균단가'] = yunrip_selected_df[['지번주소', '건축년도', '계약날짜', '지번별_전용면적별_평균단가']].groupby(['지번주소', '건축년도', '계약날짜']).transform('mean')['지번별_전용면적별_평균단가']

In [ ]:
yunrip_selected_df = yunrip_selected_df.drop(columns=['전용면적(㎡)', '지번별_전용면적별_평균단가']).drop_duplicates().reset_index(drop=True)
print(yunrip_selected_df.shape)
yunrip_selected_df.head()

In [ ]:
yunrip_selected_df['구'] = yunrip_selected_df['지번주소'].apply(lambda x: x.split(' ')[1])
yunrip_selected_df['동'] = yunrip_selected_df['지번주소'].apply(lambda x: x.split(' ')[2])
yunrip_selected_df['구동'] = yunrip_selected_df['구'] + '_' + yunrip_selected_df['동']

In [ ]:
basedir = './국토교통부_실거래가_공개시스템/땅값분석/'
yunrip_df = pd.read_csv(basedir + 'yunrip_25.csv')
commerce_df = pd.read_csv(basedir + 'commerce_25.csv')
land_df = pd.read_csv(basedir + 'land.csv')

In [ ]:
common_cols = ['시군구', '번지', '용도지역', '도로조건', '계약날짜', '대지면적(㎡)', '거래금액(만원)', '단가(㎡)', '단가(평)', '부동산유형', '건물연식']
common_df = pd.concat([yunrip_df[common_cols], commerce_df[common_cols], land_df[[col for col in land_df.columns if col not in ['지목', '지분구분', '해제사유발생일']]]])
common_df['계약날짜'] = pd.to_datetime(common_df['계약날짜'], format='%Y-%m-%d')
print(common_df.shape)
common_df.head()

In [ ]:
common_df['구'] = common_df['시군구'].apply(lambda x: x.split(' ')[1])
common_df['동'] = common_df['시군구'].apply(lambda x: x.split(' ')[2])

In [ ]:
yunrip_bulk_df['구동'] = yunrip_bulk_df['구'] + '_' + yunrip_bulk_df['동']

In [ ]:
summary_df = pd.DataFrame({'지역':[], '2종일반중간값':[], '집값중간값':[]})

count = 0
for gudong in tqdm(yunrip_selected_df['구동'].unique()):
    gu = gudong.split('_')[0]
    dong = gudong.split('_')[1]
    
    target_gu = gu
    target_dong = dong
    target_purpose = '2종일반'
    
    target_df = common_df[
        (common_df['구'] == target_gu)
        &(common_df['동'] == target_dong)
        &(common_df['용도지역'].str.contains(target_purpose))
        &(common_df['계약날짜'] >= (date_today - pd.DateOffset(months=target_period)))
        &(common_df['단가(㎡)'] > 250)
    ].sort_values(['번지'])
    
    yunrip_target_df = yunrip_selected_df[
        (yunrip_selected_df['구동'] == gudong)
    ]
    
    summary_df.loc[count, '지역'] = gudong
    summary_df.loc[count, '2종일반중간값'] = target_df['단가(㎡)'].median()
    summary_df.loc[count, '집값중간값'] = yunrip_target_df['지번별_평균단가'].median()
    
    count += 1

In [ ]:
summary_df['차이'] = summary_df['집값중간값'] - summary_df['2종일반중간값']

In [ ]:
summary_df = summary_df.sort_values(['차이'], ascending=False).reset_index(drop=True)
print(summary_df.shape)
summary_df.head()

In [ ]:
summary_df

In [ ]:
summary_df.to_excel('지역별_땅값_일괄매매_중간값.xlsx', index=False)

In [ ]:
summary_df = pd.DataFrame({'지역':[], '2종일반평균':[], '집값평균':[]})

count = 0
for gudong in tqdm(yunrip_bulk_df['구동'].unique()):
    gu = gudong.split('_')[0]
    dong = gudong.split('_')[1]
    
    target_gu = gu
    target_dong = dong
    target_purpose = '2종일반'
    
    target_df = common_df[
        (common_df['구'] == target_gu)
        &(common_df['동'] == target_dong)
        &(common_df['용도지역'].str.contains(target_purpose))
        &(common_df['계약날짜'] >= (date_today - pd.DateOffset(months=target_period)))
        &(common_df['단가(㎡)'] > 250)
    ].sort_values(['번지'])
    
    yunrip_target_df = yunrip_selected_df[
        (yunrip_selected_df['구동'] == gudong)
    ]
    
    summary_df.loc[count, '지역'] = gudong
    summary_df.loc[count, '2종일반평균'] = target_df['단가(㎡)'].mean()
    summary_df.loc[count, '집값평균'] = yunrip_target_df['지번별_평균단가'].mean()
    
    count += 1

In [ ]:
summary_df['차이'] = summary_df['집값평균'] - summary_df['2종일반평균']

In [ ]:
summary_df = summary_df.sort_values(['차이'], ascending=False).reset_index(drop=True)
print(summary_df.shape)
summary_df.head()

In [ ]:
summary_df

In [ ]:
summary_df.to_excel('지역별_땅값_일괄매매_평균.xlsx', index=False)

In [ ]:
yunrip_selected_df.head()

In [ ]:
inv_df = yunrip_selected_df[yunrip_selected_df['구동'] == '성동구_마장동']
print(inv_df.shape)
inv_df.head()

In [ ]:
def mean_prices_per_size(df):
    df = df.copy()
    
    df['크기별_평균단가'] = df[['지번주소', '건축년도', '계약날짜', '전용면적(㎡)', '단가(만원/㎡)']].groupby(['지번주소', '건축년도', '계약날짜', '전용면적(㎡)']).transform('mean')['단가(만원/㎡)']
    df['크기별_평균총액'] = df[['지번주소', '건축년도', '계약날짜', '전용면적(㎡)', '거래금액(만원)']].groupby(['지번주소', '건축년도', '계약날짜', '전용면적(㎡)']).transform('mean')['거래금액(만원)']
    df = df[['지번주소', '건축년도', '건물연식', '계약날짜', '계약날짜기준_건물연식', '전용면적(㎡)', '크기별_평균총액', '크기별_평균단가']]
    df = df.drop_duplicates().reset_index(drop=True)
    
    return df

In [ ]:
yunrip_short_df = mean_prices_per_size(yunrip_selected_df)
print(yunrip_short_df.shape)
yunrip_short_df.head()